# 02_data_cleaning.ipynb  
## Data Cleaning and Preparation

### 1. Import Required Libraries  
In this step, we load the essential libraries for data cleaning and manipulation:  
- **pandas** for tabular data structures and analysis  
- **numpy** for numerical operations  
- **pathlib** for portable file path management  


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import geopandas as gpd

### 2. Define Data Directories  
Set up input (`data/raw`) and output (`data/processed`) paths, ensuring the output folder exists before saving cleaned files.

In [2]:
RAW_DIR = Path('../data/raw')
PROC_DIR = Path('../data/processed')
PROC_DIR.mkdir(parents=True, exist_ok=True)


### 3. Load Raw Datasets  
- **Crime:** CSV file containing NYPD incident records  
- **Weather:** CSV file with daily NOAA observations  
  
Print the file paths for verification, then load both into pandas DataFrames.

In [3]:

# --- 1) Diagnostics (optional) ---
print("Working directory:", os.getcwd())
print("Raw folder contents:", os.listdir(Path('..') / 'data' / 'raw'))

# --- 2) Point to your raw data directory ---
RAW_DATA_DIR = Path('..') / 'data' / 'raw'

# --- 3) Load & concatenate all split NYPD CSVs ---
crime_files = sorted(RAW_DATA_DIR.glob('nypd_*.csv'))
print("Found crime parts:", crime_files)

crime_parts = []
for f in crime_files:
    print(f"Loading {f.name} ...")
    df_part = pd.read_csv(
        f,
        parse_dates=['cmplnt_fr_dt'],   # your lowercase date column
        low_memory=False
    )
    crime_parts.append(df_part)

df_crime = pd.concat(crime_parts, ignore_index=True)
print("Combined crime shape:", df_crime.shape)

# --- 4) Load NOAA weather file ---
df_weather = pd.read_csv(
    RAW_DATA_DIR / 'noaa_ghcnd_2024.csv',
    parse_dates=['DATE'],
    low_memory=False
)
print("Weather shape:", df_weather.shape)

# --- 5) Load boroughs GeoJSON ---
gdf_boroughs = gpd.read_file(RAW_DATA_DIR / 'nyc_boroughs.geojson')
print("Boroughs shape:", gdf_boroughs.shape)


Working directory: c:\Users\vinic\nyc-crime-climate-analysis\notebooks
Raw folder contents: ['noaa_ghcnd_2024.csv', 'nyc_boroughs.geojson', 'nypd_1.csv', 'nypd_2.csv', 'nypd_3.csv', 'nypd_4.csv']
Found crime parts: [WindowsPath('../data/raw/nypd_1.csv'), WindowsPath('../data/raw/nypd_2.csv'), WindowsPath('../data/raw/nypd_3.csv'), WindowsPath('../data/raw/nypd_4.csv')]
Loading nypd_1.csv ...
Loading nypd_2.csv ...
Loading nypd_3.csv ...
Loading nypd_4.csv ...
Combined crime shape: (565118, 35)
Weather shape: (366, 151)
Boroughs shape: (5, 5)



#### 3.1. Preview & Data Types  
- Display the first few rows of each DataFrame for a quick glance.  
- Inspect each column’s data type to determine necessary conversions.


In [4]:
## 3.1. Preview and datatypes
display(df_crime.head())  
display(df_crime.dtypes)

display(df_weather.head())
"display(df_weather.dtypes)"  # fix display below

# %%
# Fix display of df_weather.dtypes
display(df_weather.dtypes)


,cmplnt_num,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ky_cd,ofns_desc,pd_cd,...,susp_sex,transit_district,latitude,longitude,lat_lon,patrol_boro,station_name,vic_age_group,vic_race,vic_sex
0,298692973,2024-05-25,10:00:00,NaN,(null),67,2024-12-31T00:00:00.000,109,GRAND LARCENY,420.0,...,(null),NaN,40.647974,-73.953491,"(40.647974, -73.953491)",PATROL BORO BKLYN SOUTH,(null),25-44,BLACK,F
1,298695123,2024-10-28,18:00:00,2024-10-28T00:00:00.000,19:00:00,33,2024-12-31T00:00:00.000,233,SEX CRIMES,175.0,...,M,NaN,40.840781,-73.936245,"(40.8407811088581, -73.9362453818218)",PATROL BORO MAN NORTH,(null),25-44,WHITE HISPANIC,F
2,298665519,2024-10-23,00:00:00,2024-10-24T00:00:00.000,00:00:00,70,2024-12-31T00:00:00.000,109,GRAND LARCENY,425.0,...,(null),NaN,40.623789,-73.963023,"(40.623789, -73.963023)",PATROL BORO BKLYN SOUTH,(null),UNKNOWN,UNKNOWN,D
3,298692956,2024-09-21,01:00:00,2024-12-30T00:00:00.000,23:00:00,60,2024-12-31T00:00:00.000,578,HARRASSMENT 2,638.0,...,F,NaN,40.588192,-73.987785,"(40.588192, -73.987785)",PATROL BORO BKLYN SOUTH,(null),25-44,WHITE,M
4,298697962,2024-09-27,19:30:00,2024-12-28T00:00:00.000,18:16:00,83,2024-12-31T00:00:00.000,109,GRAND LARCENY,402.0,...,U,NaN,40.699565,-73.935779,"(40.699565, -73.935779)",PATROL BORO BKLYN NORTH,(null),25-44,WHITE,M


cmplnt_num                   object
cmplnt_fr_dt         datetime64[ns]
cmplnt_fr_tm                 object
cmplnt_to_dt                 object
cmplnt_to_tm                 object
addr_pct_cd                   int64
rpt_dt                       object
ky_cd                         int64
ofns_desc                    object
pd_cd                       float64
pd_desc                      object
crm_atpt_cptd_cd             object
law_cat_cd                   object
boro_nm                      object
loc_of_occur_desc            object
prem_typ_desc                object
juris_desc                   object
jurisdiction_code             int64
parks_nm                     object
hadevelopt                   object
housing_psa                 float64
x_coord_cd                  float64
y_coord_cd                  float64
susp_age_group               object
susp_race                    object
susp_sex                     object
transit_district            float64
latitude                    

,STATION,DATE,ACMC,ACMH,ACSC,ACSH,ADPT,ASLP,ASTP,AWBT,...,WT22,WV01,WV03,WV07,WV18,WV20,alt,station_info,station_name,time
0,USW00094728,2024-01-01,NaN,NaN,NaN,NaN,-11.0,10166.0,10115.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094728,2024-01-02,NaN,NaN,NaN,NaN,-61.0,10176.0,10125.0,-6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00094728,2024-01-03,NaN,NaN,NaN,NaN,-44.0,10159.0,10108.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094728,2024-01-04,NaN,NaN,NaN,NaN,-61.0,10159.0,10112.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00094728,2024-01-05,NaN,NaN,NaN,NaN,-100.0,10240.0,10190.0,-33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


STATION                 object
DATE            datetime64[ns]
ACMC                   float64
ACMH                   float64
ACSC                   float64
                     ...      
WV20                   float64
alt                    float64
station_info           float64
station_name           float64
time                   float64
Length: 151, dtype: object

### 4. Convert Date & Time Columns  
- **Crime:** Merge date (`CMPLNT_FR_DT`) and time (`CMPLNT_FR_TM`) into a single `complaint_datetime` column, and extract a separate `date` field.  
- **Weather:** Parse the `DATE` column into a standardized `date` field.


### 4.1. Convert Crime Date & Time  
Combine the separate complaint date (`CMPLNT_FR_DT`) and time (`CMPLNT_FR_TM`) fields into:
- **`complaint_datetime`**: a full `datetime` object  
- **`date`**: the calendar date extracted from that timestamp  

This ensures we can both timestamp incidents precisely and group them by day for aggregation.


In [5]:
# 1) Standardize column names (just to be safe)
df_crime.columns = df_crime.columns.str.strip().str.lower()

# 2) Combine date and time into a single datetime column
# — specify a format if you know it (e.g. format='%m/%d/%Y %H:%M:%S')
df_crime['complaint_datetime'] = pd.to_datetime(
    df_crime['cmplnt_fr_dt'].astype(str) 
    + ' ' + df_crime['cmplnt_fr_tm'].astype(str),
    errors='coerce'
)

# 3) Extract only the date portion
df_crime['date'] = df_crime['complaint_datetime'].dt.date

# 4) Verify the result
print(df_crime[['cmplnt_fr_dt','cmplnt_fr_tm','complaint_datetime','date']].head())


  cmplnt_fr_dt cmplnt_fr_tm  complaint_datetime        date
0   2024-05-25     10:00:00 2024-05-25 10:00:00  2024-05-25
1   2024-10-28     18:00:00 2024-10-28 18:00:00  2024-10-28
2   2024-10-23     00:00:00 2024-10-23 00:00:00  2024-10-23
3   2024-09-21     01:00:00 2024-09-21 01:00:00  2024-09-21
4   2024-09-27     19:30:00 2024-09-27 19:30:00  2024-09-27


### 4.2. Convert Weather Date  
Parse the NOAA `DATE` field into a proper `date` column.  
- Converts the original string (YYYY-MM-DD) into a `datetime.date` type  
- Standardizes the weather dataset’s date field for easy joins with the crime data

In [6]:
# %%
# 4.2. Convert weather date
df_weather['date'] = pd.to_datetime(df_weather['DATE'], errors='coerce').dt.date


### 5. Normalize Column Names to snake_case  
Rename all fields to a clear, consistent style:  
- e.g., `CMPLNT_FR_DT` → `complaint_date`  
- e.g., `TMAX` → `temp_max`  
- e.g., `PRCP` → `precipitation`


In [7]:
## 5. Normalize column names to snake_case

# %%
# Rename crime columns
mapping_crime = {
    'CMPLNT_FR_DT': 'complaint_date',
    'CMPLNT_FR_TM': 'complaint_time',
    'CMPLNT_TO_DT': 'complaint_to_date',
    'CMPLNT_TO_TM': 'complaint_to_time',
    'ADDR_PCT_CD': 'precinct_code',
    'KY_CD': 'offense_code',
    'OFNS_DESC': 'offense_desc',
    'BORO_NM': 'borough',
    'LAW_CODE': 'law_code',
    'LAW_CAT_CD': 'law_category',
    # add more as needed
}

df_crime = df_crime.rename(columns=mapping_crime)

# Rename weather columns
mapping_weather = {
    'DATE': 'date_recorded',
    'TMAX': 'temp_max',
    'TMIN': 'temp_min',
    'PRCP': 'precipitation',
    # add more based on your CSV
}

df_weather = df_weather.rename(columns=mapping_weather)

### 6. Handle Missing Values  
1. **Assessment:** Compute the percentage of missing values per column to guide decisions.  
2. **Imputation:**  
   - Fill missing precipitation values with `0` (interpreting absence as no rainfall).  
3. **Removal:**  
   - Drop any crime records that lack a valid datetime.

### 6.1. Calculate Missing Value Percentages in Crime Data  
- Compute the percentage of nulls for each column:  
  - `df_crime.isna().mean() * 100`  
- Sort descending to identify the most incomplete fields first  
- Display the top 10 columns by missing rate to guide cleaning decisions  

In [8]:

# %%
# 6.1. Missing percentage in crime
df_crime_missing = df_crime.isna().mean().sort_values(ascending=False) * 100
display(df_crime_missing.head(10))

transit_district    95.247010
housing_psa         93.838632
cmplnt_to_dt         5.017890
pd_cd                0.060695
latitude             0.002300
lat_lon              0.002300
longitude            0.002300
x_coord_cd           0.001593
y_coord_cd           0.001593
cmplnt_to_tm         0.000000
dtype: float64

### 6.2. Calculate Missing Value Percentages in Weather Data  
- Similarly, compute the null percentage for each weather column:  
  - `df_weather.isna().mean() * 100`  
- Sort and display the highest-missing columns  
- Compare with crime data to decide whether to impute or drop  


In [9]:
# 6.2. Missing percentage in weather
df_weather_missing = df_weather.isna().mean().sort_values(ascending=False) * 100
display(df_weather_missing.head(10))


ACMH    100.0
ACMC    100.0
ACSH    100.0
ACSC    100.0
DAEV    100.0
DAPR    100.0
DASF    100.0
AWDR    100.0
FMTM    100.0
FRGB    100.0
dtype: float64

### 6.3. Simple Imputation for Weather Precipitation  
- Interpret missing precipitation as “no rain”  
- Fill nulls in `precipitation` with `0`:  
  ```python
  df_weather['precipitation'] = df_weather['precipitation'].fillna(0)

In [10]:

# %%
# 6.3. Simple imputation example for weather
# Fill missing precipitation with 0 (assuming missing means no rain)
df_weather['precipitation'] = df_weather['precipitation'].fillna(0)

### 6.4. Drop Crime Records with Invalid Datetimes  
- Remove any rows where the combined `complaint_datetime` or its extracted `date` is null  
- Ensures all remaining crime incidents have a valid timestamp for accurate time-series analysis  
- Implemented via:  
  ```python
  df_crime = df_crime.dropna(subset=['complaint_datetime', 'date'])

In [11]:

# %%
# 6.4. Drop crime rows with invalid dates
df_crime = df_crime.dropna(subset=['complaint_datetime', 'date'])


### 7. Filter Analysis Period  
Restrict both datasets to the timeframe **2018-01-01** through **2022-12-31** to align with project scope.


In [12]:
# Define the time range for 2024
start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2024-12-31')

# Ensure the 'date' column is datetime
df_crime['date'] = pd.to_datetime(df_crime['date'], errors='coerce')

# Filter crime data for 2024
df_crime_2024 = df_crime[(df_crime['date'] >= start_date) & (df_crime['date'] <= end_date)]

# Filter weather data for 2024 (assuming 'date' column exists there too)
df_weather['date'] = pd.to_datetime(df_weather['date'], errors='coerce')
df_weather_2024 = df_weather[(df_weather['date'] >= start_date) & (df_weather['date'] <= end_date)]

# Print how many records are left
print(f"Crime records between {start_date.date()} and {end_date.date()}: {len(df_crime_2024)}")
print(f"Weather records between {start_date.date()} and {end_date.date()}: {len(df_weather_2024)}")


Crime records between 2024-01-01 and 2024-12-31: 565118
Weather records between 2024-01-01 and 2024-12-31: 366


### 8. Export Cleaned Datasets  
Save the final DataFrames as CSV files:  
- **crime_clean.csv**  
- **weather_clean.csv**  

These files will be stored in `data/processed/` and provide the foundation for subsequent EDA and modeling steps.

In [13]:
## 8. Save cleaned datasets

# %%
clean_crime_file = PROC_DIR / 'crime_clean.csv'
clean_weather_file = PROC_DIR / 'weather_clean.csv'

df_crime.to_csv(clean_crime_file, index=False)
df_weather.to_csv(clean_weather_file, index=False)

print(f"Saved cleaned crime data to: {clean_crime_file}")
print(f"Saved cleaned weather data to: {clean_weather_file}")


Saved cleaned crime data to: ..\data\processed\crime_clean.csv
Saved cleaned weather data to: ..\data\processed\weather_clean.csv
